In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
import os
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split

## Importer le dataset depuis Kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install kaggle

In [12]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list --user yannickboursier

In [ ]:
!kaggle datasets download -d yannickboursier/liver-ct-hcc -p /content/sample_data/

## Lire et analyser données en format hdf5

In [ ]:
path = '/content/drive/My Drive/Colab_Notebooks/'

In [ ]:
def read_image_mask_hdf5(path):


**Question** : Afficher une image et analyser.
Puis ajouter son masque associé

## Lire le tableau de données

In [ ]:
data = pd.read_csv(path + 'data_master_TSI.csv')

**Question** : tracer l'histogramme représentant la taille de la tumeurs

**Question** : sélectionner 2 seuils pour créer les catégories "Small", "Medium" et "Large" puis dans le csv ajouter une colonne avec la catégorie de la tumeur pour chaque image en éliminant les slides qui n'ont pas de tumeurs

**Question** : Afficher le nombre d'images par catégories

## Séparer la base de données en entraînement, validation et test

**Question** : Coder la métrique de Dice

## Modèle UNet

In [ ]:

import numpy as np
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from tensorflow.keras.models import Model


def unet(pretrained_weights = None,input_size = (512,512,1)):
    inputs = Input(input_size)
    print(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

    model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

**Question** : combien y-a-t'il de couches dans le Unet proposé ?
Quelle est la taille du kernel ?



## Générateur de batch

In [ ]:
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation
data_gen_args = dict(rotation_range=45,
                     width_shift_range=0.05,
                     height_shift_range=0.05,
                     horizontal_flip=1,
                     vertical_flip=1)

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

class DataGenerator(keras.utils.Sequence):
    def __init__(self, list_IDs, norm=False, batch_size=32, dim=(512,512), n_channels=1,
                 n_classes=1, shuffle=True):
        'Initialization'

        self.dim = dim
        self.batch_size = batch_size
        self.norm = norm
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()




    def __len__(self):
        # 'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        # 'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        # 'Updates indexes after each epoch'
        # self.indexes = np.array(self.list_IDs.index)
        self.indexes = np.arange(len(self.list_IDs))

        if self.shuffle == True:
            np.random.shuffle(self.indexes)


    def normalisation_data(self, input):
        scaler = MinMaxScaler()
        output = scaler.fit_transform(input)
        return output

    def __data_generation(self, list_IDs_temp):
        # 'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, *self.dim, self.n_channels))


        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            im, mask = read_image_mask_hdf5(ID)





            X[i, :, :, 0] = np.array(im)
            y[i, :, :, 0] = np.array(mask)

            #X[i, :, :, 0] -= X[i, :, :, 0].mean()
            #X[i, :, :, 0] /= X[i, :, :, 0].std()



        seed = np.random.randint(100000)
        image_datagen_ = image_datagen.flow(X, batch_size=self.batch_size, seed=seed)
        mask_datagen_ = mask_datagen.flow(y, batch_size=self.batch_size, seed=seed)
        X = next(image_datagen_)
        y = np.round(next(mask_datagen_))


        X -= X.mean()
        X /= X.std()




        return X, y


## Lancer un entraînement

In [ ]:
import os
import pandas as pd
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split


path_csv = path +"data_master_TSI.csv"
nb_batch = 8
nb_epochs = 20
LR = 0.001
nb_channels = 16
width = 512
height = 512



def training(path_csv, nb_batch, nb_epochs, LR, nb_channels, width, height):

    df = data[data['size']!='Not Specified']

    print(df.shape)

    # Split dataset in 3 :
    #  - train : to train each epochs
    #  - val : to validate at the end of each epoch
    #  - test : to test the model at the end with data never seen during training

    df_train, df_test = train_test_split(df, test_size = 0.1, random_state=42, shuffle=True
                                         , stratify=df['size']
    )
    df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=42, shuffle=True
                                        , stratify=df_train['size']
    )


    print(df_train.shape)
    print(df_test.shape)
    print(df_val.shape)



    list_train = list(df_train['name'])
    list_val = list(df_val['name'])

    df = pd.DataFrame(df)
    df['Dataset'] = 0
    print(df.head())

    for i, j in df.iterrows():
        if df['name'][i] in list_train :
            df['Dataset'][i] = 'train'
        elif df['name'][i] in list_val :
            df['Dataset'][i] = 'validation'
        else:
            df['Dataset'][i] = 'test'

    print(df.head())

    df_ = pd.DataFrame(df, columns=['ID_volume', 'path_volume', 'path_mask', 'original shape',
                                                  'slice number', 'path_image', 'label', 'Dataset'])
    #df_.to_csv('/imXgam_users/cannet/Deep_learning/Segmentation/Data_tumor/split_tp_stratifyBySize.csv')


    gen_train = DataGenerator(list_train, norm = False, batch_size = nb_batch, dim = (height, width),
                              n_channels = 1, n_classes = 1, shuffle = True)
    gen_val = DataGenerator(list_val, norm = False, batch_size = nb_batch, dim = (height, width),
                              n_channels = 1, n_classes = 1, shuffle = False)

    #logs_dir = os.path.join(args.save, args.model)
   # if not os.path.exists(logs_dir):
   #     os.makedirs(logs_dir)
    # Make it the current directory "."
   # os.chdir(logs_dir)


    model = unet()
    print(model.summary())


    model.compile(optimizer=keras.optimizers.Adam(args.rate),
                  loss=keras.losses.binary_crossentropy,
                  metrics=[dice_coef]
                  )

    steps_per_epochs = len(df_train) // nb_batch
    validation_steps = len(df_val) // nb_batch
    model.fit_generator(generator=gen_train, steps_per_epoch=steps_per_epochs,
                    epochs=nb_epochs,
                    validation_data=gen_val,
                    validation_steps=validation_steps,
                    #callbacks=callbacks
                        )

if __name__ == '__main__':

    path_csv = path +"data_master_TSI.csv"
    nb_batch = 8
    nb_epochs = 20
    LR = 0.001
    nb_channels = 16
    width = 512
    height = 512

    training(path_csv, nb_batch, nb_epochs, LR, nb_channels, width, height)
